## Capstone Project : Mahindra First Choice


### Problem Statement
Mahindra First Choice Services (MFCS) is a company of Mahindra Group and is India’s leading chain of multi-brand car workshops with over 335+ workshops present in 267+ towns & 24 states. It has serviced over 10,50,000 cars. The company aims to establish countrywide network of over 400 workshops by March 2018.
Mahindra would now like to leverage the data that they have and address the key issues they have. Read along to know how you can help them improve their business.
The dataset consist of three aspects:
* Customer data: where the details of the customer like the car owned, state and place of residence, order type, etc are present. Data dimension is of 534000 Customer entries
* Invoice data: where information related to customer visits and transactions are recorded, whether a customer as insurance claims, bifurcation of the amount 
 paid, for what type of service did the customer came for, etc…
* Material Inventory: where information related to what kind of service did the customer took and what kind of material was used to service, Labor information and the cost for the service, Plant and plant name where the customer took the service.


### Objective

### Geolocation Based Customer Analysis:
The idea is to explore how various factors like car make & model, time and type of service etc. vary with location. Since the servicing industry is local in nature, this kind of an analysis could possibly render some really interesting business insights.
Furthermore, this analysis will enable us to formulate more concrete machine learning problems.
From the data at hand it is possible to extract insights about customer behaviour especially the following questions can be addressed

#### Problem Statement-1
Identifying the ownership pattern of cars throughout the country. This also captures the problem wherein information regarding the spending patterns can be identified Expected Business Outcome: Mahindra First Choice Services will be benefited in multiple ways. Knowing the ownership pattern targeted marketing campaigns could be carried out. Knowing the spending patterns services could be suited to the particular spending pattern.

#### Problem Statement-2
Identify the type of order each state receives and present it as an interactive visualization. Expected Business Outcome: This could potentially give information about how Mahindra First Choice needs to be prepared to tackle various seasonal cases

### Market Segmentation:
Market segmentation is the process of dividing a market of potential customers into internally homogeneous and mutually heterogeneous groups or segments, based on different characteristics captured in the data. Groups created through such a segmentation exercise many times reveal behavioral patterns which are different from generally accepted segments by the business. The exercise is broadly known as “clustering” and is aimed at finding the consumers who will respond similarly to various stimuli by detecting underlying behavior patterns.
Though clustering falls under a Machine Learning problem category called unsupervised learning, which requires extensive efforts, it is possible to carry out a visual analysis in a relatively short timespan.

#### Problem Statement: 
Customer Lifetime value prediction - Based on Customer segments, predict the revenue that can be extracted from each segment over a life of the car -Regression/Time Series.


## Start of Notebook

#### Why are we doing this
- To Help Mahindra improve its business from analysing the data given like max. car model coming for repair location-wise, top sales garage etc
- To Identify more profitable area or expansion in other regions

#### Stakeholders:
- Marketing Head of Mahindra First [For Marketing Campaign]
- Operational Head/State Head of Mahindra First [For knowing In-Outs operation in Garage]
- Project Manager [i.e. Our Consulting Firm Manager - To which Mahindra First Company are first point of contact with for Analysis]

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
import plotly
import plotly.graph_objects as go
import re
import pgeocode

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
# If DataSet lies in another Directory
#os.getcwd()
os.chdir("C:\\Users\\Dashang\\Documents\\GreyAtom_Capstone_MahindraFirst\\") 

In [4]:
customer = pd.read_excel("Customer_Data.xlsx",index=False)

In [4]:
def clean_column_name(df):
    
    # Map the lowering function to all column names
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace('/', '').str.replace(')', '').str.replace('%', '').str.replace('.', '')
        
    return df

### Cleaning Customer DataSet

### Notes in EDA:

#### For Customer DataSet

- It is seen that column <b>Marital Status </b>, <b>Occupation</b>, <b>Date of Birth</b>, <b>Death Date</b> has more than <b>97% of null values</b>. Its hard to Impute values in it.
- Column <b>Title</b> is of less significance i.e its just salutation hence dropping the same. We treat customer as customer whether is male or female. No Gender Inequality or Professional Inequality.
- Column <b>Partner_Type</b> and <b>Data Origin</b> has less null value hence using Imputer Library to Impute Missing Values using 'Most-Frequent Strategy.

In [6]:
def CleanCustomerDf(df):
    
    df = df.drop(['marital_status','occupation','date_of_birth','death_date','title'],axis=1)
    imp = SimpleImputer(missing_values=np.nan, strategy="most_frequent")
    df.data_origin=imp.fit_transform(df[["data_origin"]]).ravel()
    df.partner_type=imp.fit_transform(df[["partner_type"]]).ravel()
    
    return df

In [7]:
customer.isnull().sum() / customer.shape[0] * 100.00

Business Partner     0.000000
Customer No.         0.000000
Partner Type         0.014046
Data Origin          0.024490
Title                8.338525
Marital Status      97.112209
Occupation          97.378533
Date of Birth       97.579132
Death date          99.880613
dtype: float64

In [8]:
customer = clean_column_name(customer)
customer = CleanCustomerDf(customer)

In [9]:
customer.isnull().sum() / customer.shape[0] * 100.00

business_partner    0.0
customer_no         0.0
partner_type        0.0
data_origin         0.0
dtype: float64

In [10]:
customer.to_csv('customer_cleaned.csv', index=False)

### Cleaning JTD (inventory) DataSet

### Notes in EDA:

#### For JTD (Inventory) DataSet

- Column <b>material</b>, <b>Labor Value Number</b> and <b>description</b> has been filled with 'Not Mentioned' to counter Nan 
- Column <b>Target Quantity UOM </b> has small fraction of missing values. Filtering item_category with value 'G2TX' shows order quantity also as Nan so unable to determine Target Quantity UOM hence using Imputer Library to Impute Missing Values using 'Most-Frequent Strategy.


In [11]:
jtd = pd.read_csv("JTD.csv",index_col=0)

C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [12]:
def CleanJtdDf(df):
    
    df['material']=df['material'].fillna('Not_Mentioned')
    df['labor_value_number']=df['labor_value_number'].fillna('Not_Mentioned')
    df['description']=df['description'].fillna('Not_Mentioned')
    
    #Impute based on Most Frequent
    #imp = SimpleImputer(missing_values=np.nan, strategy="most_frequent")
    #df.target_quantity_uom=imp.fit_transform(df[["target_quantity_uom"]]).ravel()
    
    #OR
    df['target_quantity_uom']=df['target_quantity_uom'].fillna('EA')
    
    return df

In [13]:
jtd.isnull().sum() / jtd.shape[0] * 100.00

DBM Order               0.000000
Order Item              0.000000
Material                0.918750
Labor Value Number     67.159778
Description             0.918750
Item Category           0.000000
Order Quantity          0.000000
Target quantity UoM     0.918750
Net value               0.000000
dtype: float64

In [36]:
jtd.head()

NameError: name 'jtd' is not defined

In [15]:
jtd = clean_column_name(jtd)
jtd = CleanJtdDf(jtd)

In [16]:
jtd.isnull().sum() / jtd.shape[0] * 100.00

dbm_order              0.0
order_item             0.0
material               0.0
labor_value_number     0.0
description            0.0
item_category          0.0
order_quantity         0.0
target_quantity_uom    0.0
net_value              0.0
dtype: float64

In [17]:
jtd.to_csv('jtd_cleaned.csv', index=False)

In [ ]:
jtd.head()

### Cleaning Invoice DataSet

### Notes in EDA:

#### For Invoice DataSet

- Column <b>material</b>, <b>Labor Value Number</b> and <b>description</b> has been filled with 'Not Mentioned' to counter Nan 
- Column <b>Target Quantity UOM </b> has small fraction of missing values. Filtering item_category with value 'G2TX' shows order quantity also as Nan so unable to determine Target Quantity UOM hence using Imputer Library to Impute Missing Values using 'Most-Frequent Strategy.


In [ ]:
def GetAreaFromPin(pin):
    nomi = pgeocode.Nominatim('in')
    nomi.query_postal_code(pin)
    nomi.place_name
    return area

In [ ]:
def GetLatFromPin(pin):
    nomi = pgeocode.Nominatim('in')
    nomi.query_postal_code(pin)
    return nomi.latitude
    

In [ ]:
def GetLonFromPin(pin):
    nomi = pgeocode.Nominatim('in')
    nomi.query_postal_code(pin)
    return nomi.longitude
    

In [ ]:
def GetDistrictFromPin(pin):
    nomi = pgeocode.Nominatim('in')
    nomi.query_postal_code(pin)
    return nomi.county_name
    

In [ ]:
def GetStateFromPin(pin):
    nomi = pgeocode.Nominatim('in')
    nomi.query_postal_code(pin)
    return nomi.state_name
    

In [161]:
invoice.head()

,amt_rcvd_from_custom,amt_rcvd_from_ins_co,area__locality,cgst14,cgst25,cgst6,cgst9,city,cash_cashless_type,claim_no,cust_type,customer_no,district,expiry_date,gate_pass_date,gate_pass_time,igst12,igst18,igst28,igst5,insurance_company,invoice_date,invoice_no,invoice_time,job_card_no,jobcard_date,jobcard_time,kms_reading,labour_total,make,misc_total,model,odn_no,osl_total,order_type,outstanding_amt,parts_total,pin_code,plant,plant_name1,policy_no,print_status,recovrbl_exp,regn_no,sgstugst14,sgstugst25,sgstugst6,sgstugst9,service_advisor_name,tds_amount,technician_name,total_amt_wtd_tax,total_cgst,total_gst,total_igst,total_sgstugst,total_value,user_id
0,0.0,0.0,MAJIWADA,0.0,0.0,0.0,0.0,Thane,NaN,NaN,Retail,67849,Maharashtra,NaN,NaN,00:00:00,0.0,0.0,0.0,0.0,NaN,2015-01-02,7005200002,11:30:36,168303,2014-12-13,14:29:43,49317,1203.14,GENERAL MOTORS,0.00,SPARK,7.005200e+09,500.06,Paid Service,0.0,2348.75,400601,BC01,THANE,NaN,NO,0.0,KA19MA1291,0.0,0.0,0.0,0.0,NaN,0.0,RUPESH,4051.95,0.0,0.0,0.0,0.0,4051.95,BC01FS1
1,0.0,0.0,THNAE,0.0,0.0,0.0,0.0,THNAE,NaN,NaN,Retail,84419,Maharashtra,NaN,NaN,00:00:00,0.0,0.0,0.0,0.0,NaN,2015-01-03,7005200003,10:07:32,173997,2015-01-02,14:12:18,78584,804.26,TATA MOTORS,197.03,INDICA,7.005200e+09,0.00,SMC Value Package,0.0,0.00,400603,BC01,THANE,NaN,NO,0.0,MH43R3046,0.0,0.0,0.0,0.0,NaN,0.0,PRASHANT,1001.29,0.0,0.0,0.0,0.0,1001.29,BC01SA2
2,0.0,0.0,THANE,0.0,0.0,0.0,0.0,THANE[W],NaN,NaN,Retail,81055,Maharashtra,NaN,NaN,00:00:00,0.0,0.0,0.0,0.0,NaN,2015-01-03,7005200004,11:12:57,173889,2015-01-02,11:40:44,33985,180.19,MARUTI SUZUKI,0.00,ZEN,7.005200e+09,0.00,Running Repairs,0.0,52.95,400607,BC01,THANE,NaN,NO,0.0,AP09AX0582,0.0,0.0,0.0,0.0,NaN,0.0,IMRAN,233.14,0.0,0.0,0.0,0.0,233.14,BC01SA2
3,0.0,0.0,THNAE,0.0,0.0,0.0,0.0,THNAE,NaN,NaN,Retail,84419,Maharashtra,NaN,NaN,00:00:00,0.0,0.0,0.0,0.0,NaN,2015-01-03,7005200005,11:40:44,174158,2015-01-03,10:12:32,78584,0.00,TATA MOTORS,0.00,INDICA,7.005200e+09,0.00,SMC Redemption,0.0,0.00,400603,BC01,THANE,NaN,NO,0.0,MH43R3046,0.0,0.0,0.0,0.0,NaN,0.0,PRASHANT,0.00,0.0,0.0,0.0,0.0,0.00,BC01SA2
4,0.0,0.0,THANE,0.0,0.0,0.0,0.0,THANE,NaN,NaN,Retail,18980,Maharashtra,NaN,NaN,00:00:00,0.0,0.0,0.0,0.0,NaN,2015-01-03,7005200006,12:07:17,173860,2015-01-02,10:45:42,50057,1202.97,HYUNDAI,100.00,SANTRO XING,7.005200e+09,499.99,Paid Service,0.0,944.16,400603,BC01,THANE,NaN,NO,0.0,MH04CD9768,0.0,0.0,0.0,0.0,NaN,0.0,DAYANAND,2747.12,0.0,0.0,0.0,0.0,2747.12,BC01SA2


In [165]:
inv = invoice[(invoice['make']=='FORD') & (invoice['model'].notna())]

In [176]:
inv['model'].value_counts().index[0]

'FIGO'

In [ ]:
def GetModelFromOtherData(make):
    result = invoice[(invoice['make']=='FORD') & (invoice['model'].notna())]
    return result['model'].value_counts().index[0]

In [5]:
invoice = pd.read_csv("Final_Invoice.csv",index_col=0)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (9,10,14,15,21,41,51) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
invoice = clean_column_name(invoice)

In [ ]:
nomi = pgeocode.Nominatim('in')

In [37]:
invoice.dtypes

amt_rcvd_from_custom    float64
amt_rcvd_from_ins_co    float64
area__locality           object
cgst14                  float64
cgst25                  float64
cgst6                   float64
cgst9                   float64
city                     object
cash_cashless_type       object
claim_no                 object
cust_type                object
customer_no              object
district                 object
expiry_date              object
gate_pass_date           object
gate_pass_time           object
igst12                  float64
igst18                  float64
igst28                  float64
igst5                   float64
insurance_company        object
invoice_date             object
invoice_no                int64
invoice_time             object
job_card_no               int64
jobcard_date             object
jobcard_time             object
kms_reading               int64
labour_total            float64
make                     object
misc_total              float64
model   

In [7]:
def CleanInvoice(df):
    
    #dropping gstcomponents just keeping final price
    df=df.drop(['odn_no','tds_amount','total_value','amt_rcvd_from_custom','amt_rcvd_from_ins_co','cgst14','cgst25','cgst6','cgst9','igst12','igst18','igst28','igst5','sgstugst14','sgstugst25','sgstugst6','sgstugst9','total_cgst','total_gst','total_igst','total_sgstugst'],axis=1) 
    
    #Remove Column having null values greater than 55%
    df=df.drop(['cash_cashless_type','claim_no','expiry_date','gate_pass_date','insurance_company','policy_no','service_advisor_name'],axis=1)
    
    
    df = df.apply(lambda x: x.str.lower() if(x.dtype == 'object') else x)
    
    #filling Null values
    df['outstanding_amt']= df['outstanding_amt'].fillna(0)
    df['regn_no'] = df['regn_no'].fillna('none')
    df['technician_name']=df['technician_name'].fillna('NoName')
    
        
    #getting area
    df['area__locality'] = df['pincode'].apply(lambda x: GetAreaFromPin(x) )
    #getting latitude
    df['latitude'] = df['pincode'].apply(lambda x: GetLatFromPin(x) )
    #getting longitude
    df['longitude'] = df['pincode'].apply(lambda x: GetLonFromPin(x) )
    
    #correcting District
    df['District'] = df['pincode'].apply(lambda x: GetDistrictFromPin(x) )
    
    #correcting State
    df['State'] = df['pincode'].apply(lambda x: GetStateFromPin(x) )
    
    
    #fill model
    df['model'] = df['make'].apply(lambda x: GetModelFromOtherData(x) )
    
    return df

In [8]:
temp=invoice.drop(['cgst14','cgst25','cgst6','cgst9','igst12','igst18','igst28','igst5','sgstugst14','sgstugst25','sgstugst6','sgstugst9','total_cgst','total_gst','total_igst','total_sgstugst'],axis=1) 


In [9]:
temp=temp.drop(['cash_cashless_type','claim_no','expiry_date','gate_pass_date','insurance_company','policy_no','service_advisor_name'],axis=1)

In [14]:
temp=temp.drop(['amt_rcvd_from_custom','amt_rcvd_from_ins_co'],axis=1)

In [15]:
temp.isnull().sum() / temp.shape[0] * 100.00

area__locality        4.852594
city                  0.000203
cust_type             0.000000
customer_no           0.000000
district              0.000000
gate_pass_time        0.000000
invoice_date          0.000000
invoice_no            0.000000
invoice_time          0.000000
job_card_no           0.000000
jobcard_date          0.000000
jobcard_time          0.000000
kms_reading           0.000000
labour_total          0.000000
make                  0.000000
misc_total            0.000000
model                 0.309762
odn_no               40.073205
osl_total             0.000000
order_type            0.000000
outstanding_amt      40.073205
parts_total           0.000000
pin_code              0.000000
plant                 0.000000
plant_name1           0.000000
print_status          0.000000
recovrbl_exp          0.000000
regn_no               0.003250
tds_amount           40.073205
technician_name      48.106290
total_amt_wtd_tax     0.000000
total_value          40.073205
user_id 

In [16]:
temp[temp['area__locality'].isna()]

,area__locality,city,cust_type,customer_no,district,gate_pass_time,invoice_date,invoice_no,invoice_time,job_card_no,jobcard_date,jobcard_time,kms_reading,labour_total,make,misc_total,model,odn_no,osl_total,order_type,outstanding_amt,parts_total,pin_code,plant,plant_name1,print_status,recovrbl_exp,regn_no,tds_amount,technician_name,total_amt_wtd_tax,total_value,user_id
20,NaN,Thane,Retail,E12734,Maharashtra,00:00:00,2015-01-03,7005200022,19:40:26,174232,2015-01-03,11:34:27,53311,1845.00,MARUTI SUZUKI,50.00,ALTO,7.005200e+09,0.00,Paid Service,0.0,1562.27,400607,BC01,THANE,NO,0.0,MH43V1363,0.0,RUPESH,3457.27,3457.27,BC01SA1
24,NaN,Thane,Retail,E6634,Maharashtra,00:00:00,2015-01-04,7005200026,14:16:04,173858,2015-01-02,10:22:38,79527,1850.03,TATA MOTORS,100.00,INDICA,7.005200e+09,0.00,Running Repairs,0.0,6737.75,400602,BC01,THANE,NO,0.0,MH01MA9103,0.0,PRASHANT,8687.78,8687.78,BC01SA2
42,NaN,CHUNABHATTI,Retail,536,Maharashtra,00:00:00,2015-01-08,7005200044,15:12:40,175322,2015-01-07,10:08:37,64831,1203.06,HYUNDAI,50.00,SANTRO,7.005200e+09,1573.07,Paid Service,0.0,1615.62,400022,BC01,THANE,NO,0.0,MH03AF5463,0.0,Ananta,4441.75,4441.75,BC01SA1
54,NaN,Thane WEST,Retail,E3930,Maharashtra,00:00:00,2015-01-10,7005200056,11:00:28,176295,2015-01-10,10:56:49,74178,4166.49,GENERAL MOTORS,276.94,AVEO,7.005200e+09,0.00,SMC Value Package,0.0,3556.77,400607,BC01,THANE,NO,0.0,MH02JP8453,0.0,HARSHAD,8000.20,8000.20,BC01SA1
70,NaN,Thane WEST,Retail,E3930,Maharashtra,00:00:00,2015-01-11,7005200072,16:23:59,176332,2015-01-10,11:22:49,74178,3458.91,GENERAL MOTORS,199.99,AVEO,7.005200e+09,0.00,Running Repairs,0.0,14039.84,400607,BC01,THANE,NO,0.0,MH02JP8453,0.0,HARSHAD,17698.74,17698.74,BC01SA1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
982793,NaN,Nashik,Retail,E14068,Maharashtra,00:00:00,2016-06-30,7169800516,21:23:13,400761,2016-06-30,15:16:48,47535,400.01,MARUTI SUZUKI,0.00,WAGONR,NaN,0.00,Running Repairs,NaN,1962.65,422000,X099,X099 - AUTO WORLD CAR CARE,NO,0.0,MH15DC1064,NaN,NaN,2362.66,NaN,X099SA1
983110,NaN,Chennai,Corporate- M&M,MFCCM19,Gujarat,00:00:00,2016-06-30,7172001384,20:33:51,396802,2016-06-25,11:53:40,120609,11650.20,MAHINDRA & MAHINDRA,300.00,BOLERO,NaN,0.00,Paid Service,NaN,10238.73,600017,X110,X110 - KALYAN SERVICES,NO,0.0,TN66F2961,NaN,NaN,22188.93,NaN,X110SA1
984138,NaN,Nashik,Retail,E12096,Maharashtra,00:00:00,2016-06-30,7193600049,09:28:49,397161,2016-06-25,15:29:18,100549,0.84,HYUNDAI,0.00,VERNA,NaN,0.00,Paid Service,NaN,0.00,422007,X218,X218 - PRAGATI MOTORS,NO,0.0,MH15CD961,NaN,NaN,0.84,NaN,X218SA1
984165,NaN,Nashik,Retail,E22194,Maharashtra,00:00:00,2016-06-30,7193600077,19:58:04,398704,2016-06-28,11:46:16,106470,0.84,FIAT,0.00,PALIO,NaN,0.00,Paid Service,NaN,0.00,422000,X218,X218 - PRAGATI MOTORS,NO,0.0,MH19Q7765,NaN,NaN,0.84,NaN,X218SA1


In [27]:
temp.head(100)

,amt_rcvd_from_custom,amt_rcvd_from_ins_co,area__locality,city,cust_type,customer_no,district,gate_pass_time,invoice_date,invoice_no,invoice_time,job_card_no,jobcard_date,jobcard_time,kms_reading,labour_total,make,misc_total,model,odn_no,osl_total,order_type,outstanding_amt,parts_total,pin_code,plant,plant_name1,print_status,recovrbl_exp,regn_no,tds_amount,technician_name,total_amt_wtd_tax,total_value,user_id
0,0.0,0.0,MAJIWADA,Thane,Retail,67849,Maharashtra,00:00:00,2015-01-02,7005200002,11:30:36,168303,2014-12-13,14:29:43,49317,1203.14,GENERAL MOTORS,0.00,SPARK,7.005200e+09,500.06,Paid Service,0.0,2348.75,400601,BC01,THANE,NO,0.0,KA19MA1291,0.0,RUPESH,4051.95,4051.95,BC01FS1
1,0.0,0.0,THNAE,THNAE,Retail,84419,Maharashtra,00:00:00,2015-01-03,7005200003,10:07:32,173997,2015-01-02,14:12:18,78584,804.26,TATA MOTORS,197.03,INDICA,7.005200e+09,0.00,SMC Value Package,0.0,0.00,400603,BC01,THANE,NO,0.0,MH43R3046,0.0,PRASHANT,1001.29,1001.29,BC01SA2
2,0.0,0.0,THANE,THANE[W],Retail,81055,Maharashtra,00:00:00,2015-01-03,7005200004,11:12:57,173889,2015-01-02,11:40:44,33985,180.19,MARUTI SUZUKI,0.00,ZEN,7.005200e+09,0.00,Running Repairs,0.0,52.95,400607,BC01,THANE,NO,0.0,AP09AX0582,0.0,IMRAN,233.14,233.14,BC01SA2
3,0.0,0.0,THNAE,THNAE,Retail,84419,Maharashtra,00:00:00,2015-01-03,7005200005,11:40:44,174158,2015-01-03,10:12:32,78584,0.00,TATA MOTORS,0.00,INDICA,7.005200e+09,0.00,SMC Redemption,0.0,0.00,400603,BC01,THANE,NO,0.0,MH43R3046,0.0,PRASHANT,0.00,0.00,BC01SA2
4,0.0,0.0,THANE,THANE,Retail,18980,Maharashtra,00:00:00,2015-01-03,7005200006,12:07:17,173860,2015-01-02,10:45:42,50057,1202.97,HYUNDAI,100.00,SANTRO XING,7.005200e+09,499.99,Paid Service,0.0,944.16,400603,BC01,THANE,NO,0.0,MH04CD9768,0.0,DAYANAND,2747.12,2747.12,BC01SA2
5,0.0,0.0,THANE,THANE,Retail,84457,Maharashtra,00:00:00,2015-01-03,7005200007,12:10:08,174053,2015-01-02,15:15:10,13122,0.00,TOYOTA,0.00,INNOVA,7.005200e+09,851.70,Running Repairs,0.0,14.22,400603,BC01,THANE,NO,0.0,MH04GM2960,0.0,DAYANAND,865.92,865.92,BC01SA2
6,0.0,0.0,THANE WEST,THANE,Retail,12523,Maharashtra,00:00:00,2015-01-03,7005200008,12:12:31,173953,2015-01-02,12:15:55,26964,0.00,HONDA,0.00,CITY,7.005200e+09,749.42,Running Repairs,0.0,0.00,400607,BC01,THANE,NO,0.0,MH06AL1935,0.0,PRASHANT,749.42,749.42,BC01SA2
7,0.0,0.0,G.B. ROAD,THANE,Retail,55220,Maharashtra,00:00:00,2015-01-03,7005200009,12:18:54,174254,2015-01-03,12:02:32,12943,0.00,MARUTI SUZUKI,0.00,SWIFT DZIRE,7.005200e+09,0.00,Running Repairs,0.0,0.00,400606,BC01,THANE,NO,0.0,MH04FR8641,0.0,SAMEER,0.00,0.00,BC01SA2
8,0.0,0.0,THANE,THANE[W],Retail,84511,Maharashtra,00:00:00,2015-01-03,7005200010,13:07:27,174162,2015-01-03,10:21:26,18563,0.00,TATA MOTORS,0.00,INDIGO,7.005200e+09,0.00,Running Repairs,0.0,0.00,400607,BC01,THANE,NO,0.0,MH05BJ558,0.0,HARSHAD,0.00,0.00,BC01SA1
9,0.0,0.0,THANE,THANE,Retail,84457,Maharashtra,00:00:00,2015-01-03,7005200011,15:54:21,174203,2015-01-03,11:11:00,5443,698.92,MARUTI SUZUKI,49.99,ALTO,7.005200e+09,499.95,Paid Service,0.0,2663.75,400603,BC01,THANE,NO,0.0,GA07E3439,0.0,DAYANAND,3912.61,3912.61,BC01SA1


In [13]:
invoice.isnull().sum() / invoice.shape[0] * 100.00

amt_rcvd_from_custom     40.073205
amt_rcvd_from_ins_co     40.073205
area__locality            4.852594
cgst14                   40.073205
cgst25                   40.073205
cgst6                    40.073205
cgst9                    40.073205
city                      0.000203
cash_cashless_type       94.430384
claim_no                 99.024200
cust_type                 0.000000
customer_no               0.000000
district                  0.000000
expiry_date              95.484386
gate_pass_date           91.689247
gate_pass_time            0.000000
igst12                   40.073205
igst18                   40.073205
igst28                   40.073205
igst5                    40.073205
insurance_company        95.208140
invoice_date              0.000000
invoice_no                0.000000
invoice_time              0.000000
job_card_no               0.000000
jobcard_date              0.000000
jobcard_time              0.000000
kms_reading               0.000000
labour_total        

In [146]:
invoice[invoice['model'].isna()]

,amt_rcvd_from_custom,amt_rcvd_from_ins_co,area__locality,cgst14,cgst25,cgst6,cgst9,city,cash_cashless_type,claim_no,cust_type,customer_no,district,expiry_date,gate_pass_date,gate_pass_time,igst12,igst18,igst28,igst5,insurance_company,invoice_date,invoice_no,invoice_time,job_card_no,jobcard_date,jobcard_time,kms_reading,labour_total,make,misc_total,model,odn_no,osl_total,order_type,outstanding_amt,parts_total,pin_code,plant,plant_name1,policy_no,print_status,recovrbl_exp,regn_no,sgstugst14,sgstugst25,sgstugst6,sgstugst9,service_advisor_name,tds_amount,technician_name,total_amt_wtd_tax,total_cgst,total_gst,total_igst,total_sgstugst,total_value,user_id
444,0.0,0.0,PUNE,0.0,0.0,0.0,0.0,pune,NaN,NaN,Retail,31424,Maharashtra,NaN,NaN,00:00:00,0.0,0.0,0.0,0.0,NaN,2015-01-20,7006200163,16:05:04,177600,2015-01-14,12:29:14,114532,2116.00,MAHINDRA & MAHINDRA,200.00,NaN,7.006200e+09,1525.00,Paid Service,0.0,10852.66,411021,BC02,PUNE,NaN,NO,0.0,MH12FZ2739,0.0,0.0,0.0,0.0,NaN,0.0,rajesh,14693.66,0.0,0.0,0.0,0.0,14693.66,BC02SA1
464,0.0,0.0,BANER,0.0,0.0,0.0,0.0,PUNE,NaN,NaN,Retail,61456,Maharashtra,NaN,NaN,00:00:00,0.0,0.0,0.0,0.0,NaN,2015-01-23,7006200183,11:51:21,179792,2015-01-22,13:29:36,43540,1479.02,MAHINDRA & MAHINDRA,525.00,NaN,7.006200e+09,0.00,Running Repairs,0.0,287.36,411056,BC02,PUNE,NaN,NO,0.0,MH12GK7330,0.0,0.0,0.0,0.0,NaN,0.0,rajesh,2291.38,0.0,0.0,0.0,0.0,2291.38,BC02SA2
1072,0.0,0.0,"SATPUR,",0.0,0.0,0.0,0.0,NASHIK,NaN,NaN,Retail,2287,Maharashtra,NaN,NaN,00:00:00,0.0,0.0,0.0,0.0,NaN,2015-01-29,7007200531,18:05:18,181151,2015-01-27,18:10:22,66047,899.08,FORD,0.00,NaN,7.007201e+09,0.00,Paid Service,0.0,1682.50,422001,BC03,NASHIK,NaN,NO,0.0,MH09BB5525,0.0,0.0,0.0,0.0,NaN,0.0,balu/jamesh,2581.58,0.0,0.0,0.0,0.0,2581.58,BC03SA1
1513,0.0,0.0,CHETPET,0.0,0.0,0.0,0.0,CHETPET CHENNAI,NaN,NaN,Retail,37945,Tamil Nadu,NaN,NaN,00:00:00,0.0,0.0,0.0,0.0,NaN,2015-01-09,7010200069,16:58:30,175991,2015-01-09,10:43:18,46726,0.00,HONDA,50.02,NaN,7.010200e+09,1200.48,Running Repairs,0.0,0.00,600031,BC06,GUINDY,NaN,NO,0.0,AP9BN747,0.0,0.0,0.0,0.0,NaN,0.0,karthick,1250.50,0.0,0.0,0.0,0.0,1250.50,BC06SA2
1527,0.0,0.0,CHENNAI,0.0,0.0,0.0,0.0,CHENNAI,NaN,NaN,Retail,23415,Tamil Nadu,NaN,NaN,00:00:00,0.0,0.0,0.0,0.0,NaN,2015-01-10,7010200083,19:00:16,176270,2015-01-10,10:39:33,19220,1299.19,HYUNDAI,50.01,NaN,7.010200e+09,0.00,Running Repairs,0.0,332.82,600042,BC06,GUINDY,NaN,NO,0.0,TN22BM9921,0.0,0.0,0.0,0.0,NaN,0.0,sathya,1682.02,0.0,0.0,0.0,0.0,1682.02,BC06SA2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
968280,NaN,NaN,DHABA,NaN,NaN,NaN,NaN,NAGPUR,NaN,NaN,Retail,194705,Maharashtra,NaN,NaN,00:00:00,NaN,NaN,NaN,NaN,NaN,2016-06-06,7166001424,15:57:15,384784,2016-06-06,11:30:20,160025,46.03,TATA MOTORS,0.00,NaN,NaN,0.00,Running Repairs,NaN,0.00,440023,X080,X080 - KHAMLA MOTORS,NaN,NO,0.0,MH31CV6668,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46.03,NaN,NaN,NaN,NaN,NaN,X080SA1
971105,NaN,NaN,NAGPUR,NaN,NaN,NaN,NaN,nagpur,NaN,NaN,Retail,42659,Maharashtra,NaN,2016-06-13 00:00:00,18:20:25,NaN,NaN,NaN,NaN,NaN,2016-06-13,7030205933,18:18:40,388919,2016-06-13,10:47:28,50539,1361.92,MAHINDRA & MAHINDRA,99.99,NaN,NaN,2039.88,Paid Service,NaN,2783.01,440008,BC26,NAGPUR,NaN,YES,0.0,MH31DV3633,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6284.80,NaN,NaN,NaN,NaN,NaN,BC26SA3
974495,NaN,NaN,MEDAVAKKAM,NaN,NaN,NaN,NaN,CHENNAI,NaN,NaN,Retail,45328,Tamil Nadu,2016-07-23 00:00:00,2016-06-22 00:00:00,11:17:31,NaN,NaN,NaN,NaN,BHARTI AXA GENRERAL INSURANCE,2016-06-18,7011203965,19:15:29,368596,2016-05-11,08:58:38,98424,0.00,MAHINDRA & MAHINDRA,0.00,NaN,NaN,199.70,Accidental,NaN,0.00,600025,BC07,NERKUNDRAM,B589474,YES,0.0,TN22CS111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,199.70,NaN,NaN,NaN,NaN,NaN,BC07FS1
978858,NaN,NaN,KHAKHANA,NaN,NaN,NaN,NaN,JHAJJAR,NaN,NaN,Retail,150315,Haryana,2016-04-30 00:00:00,NaN,00:00:00,NaN,NaN,NaN,NaN,IC

In [17]:
plant = pd.read_excel("Plant Master.xlsx",index=False)

In [21]:
plant.head()
#plant[plant['Plant'] == "BC01"]

,Plant,Name 1,Valuation Area,Customer no. - plant,Vendor number plant,Factory calendar,Name 2,House number and street,PO Box,Postal Code,City,Sales organization,State
0,BC07,NERKUNDRAM,BC07,PLBC07,PLBC07,IN,NaN,"No.117 Poonamallee High Road,",600107.0,600107,Chennai,MFCS,Tamil Nadu
1,BC30,SAS NAGAR - MOHALI,BC30,PLBC30,PLBC30,IN,NaN,"C-118, Phase 07,",160055.0,160055,Mohali,MFCS,Punjab
2,X001,X001 - AUTO PLANET WHEELS,X001,PLX001,NaN,IN,Thane,"Unit No.3, Kothari Compound,",400610.0,400610,Thane,MFCA,Maharashtra
3,X002,X002 - BHANDARI INDUSTRIES PVT,X002,PLX002,NaN,IN,Ghaziabad,"18/1, C Site 4, Sahibabad",201010.0,201010,Ghaziabad,MFCA,Uttar Pradesh
4,X003,X003 - Om Sri Satya Sai Automo,X003,PLX003,NaN,IN,Bengaluru,"#75, Sy.No. 83/2, Panathur Mai",560087.0,560087,Bengaluru,MFCB,Karnataka


In [128]:
pindf = pd.read_csv("all_india_PO_ver2.csv")

In [129]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [136]:
def CleanPinDataSet(df):
    
    nomi = pgeocode.Nominatim('in')
    df=clean_column_name(df)
    df = df.apply(lambda x: x.str.lower() if(x.dtype == 'object') else x)
    
    #Getting Area names
#     #df['area']= df['officename'].apply(lambda x: re.sub('([a-z])\w+',' ',x))
#     df['area'] =df['officename'].apply(lambda x: re.sub('[a-z]+[-.]+[a-z]',' ',x))
#     df['area'] =df['area'].apply(lambda x: re.sub('[0-9]',' ',x))
#     df['area'] = df['area'].apply(lambda x: "".join([char.lower() for char in x if char not in string.punctuation]))
#     #replacing double whitespace with single 
#     df['area']= df['area'].apply(lambda x: re.sub(' +', ' ',x))
    
#     #getting area
#     df['area'] = df['pincode'].apply(lambda x: GetAreaFromPin(x) )
#     #getting latitude
#     df['latitude'] = df['pincode'].apply(lambda x: GetLatFromPin(x) )
#     #getting longitude
#     df['longitude'] = df['pincode'].apply(lambda x: GetLonFromPin(x) )
    
#     #correcting District
#     df['District'] = df['pincode'].apply(lambda x: GetDistrictFromPin(x) )
    
#     #correcting State
#     df['state'] = df['pincode'].apply(lambda x: GetStateFromPin(x) )
    
    
    
    return df

In [131]:
print(pindf.columns)
pindf = CleanPinDataSet(pindf)
print(pindf.columns)

Index(['officename', 'pincode', 'officeType', 'Deliverystatus', 'divisionname', 'regionname', 'circlename', 'Taluk', 'Districtname', 'statename', 'Telephone', 'Related Suboffice', 'Related Headoffice'], dtype='object')
Index(['officename', 'pincode', 'officetype', 'deliverystatus', 'divisionname', 'regionname', 'circlename', 'taluk', 'districtname', 'statename', 'telephone', 'related_suboffice', 'related_headoffice', 'area'], dtype='object')


In [132]:
pindf.iloc[82261]

officename            ulhasnagar-4 s.o
pincode                         421004
officetype                         s.o
deliverystatus                delivery
divisionname             thane central
regionname                      mumbai
circlename                 maharashtra
taluk                           kalyan
districtname                     thane
statename                  maharashtra
telephone                 0251-2585928
related_suboffice                  NaN
related_headoffice     kalyan city h.o
area                       ulhasnagar 
Name: 82261, dtype: object

In [153]:
nomi = pgeocode.Nominatim('in')
res=nomi.query_postal_code("410506")
print(res)

postal_code                                                  410506
country code                                                     IN
place_name        Talegaon Dabhade, Adhale BK, Dhamane, Chandkhe...
state_name                                              Maharashtra
state_code                                                       16
county_name                                                    Pune
county_code                                                     521
community_name                                                Maval
community_code                                                  NaN
latitude                                                     18.735
longitude                                                    73.763
accuracy                                                          4
Name: 0, dtype: object


In [143]:
len(invoice.model.unique())

246

In [144]:
invoice.columns


Index(['amt_rcvd_from_custom', 'amt_rcvd_from_ins_co', 'area__locality', 'cgst14', 'cgst25', 'cgst6', 'cgst9', 'city', 'cash_cashless_type', 'claim_no', 'cust_type', 'customer_no', 'district', 'expiry_date', 'gate_pass_date', 'gate_pass_time', 'igst12', 'igst18', 'igst28', 'igst5', 'insurance_company', 'invoice_date', 'invoice_no', 'invoice_time', 'job_card_no', 'jobcard_date', 'jobcard_time', 'kms_reading', 'labour_total', 'make', 'misc_total', 'model', 'odn_no', 'osl_total', 'order_type', 'outstanding_amt', 'parts_total', 'pin_code', 'plant', 'plant_name1', 'policy_no', 'print_status', 'recovrbl_exp', 'regn_no', 'sgstugst14', 'sgstugst25', 'sgstugst6', 'sgstugst9', 'service_advisor_name', 'tds_amount', 'technician_name', 'total_amt_wtd_tax', 'total_cgst', 'total_gst', 'total_igst', 'total_sgstugst', 'total_value', 'user_id'], dtype='object')

In [108]:
pindf.head()

,officename,pincode,officetype,deliverystatus,divisionname,regionname,circlename,taluk,districtname,statename,telephone,related_suboffice,related_headoffice,area
0,achalapur b.o,504273,b.o,delivery,adilabad,hyderabad,andhra pradesh,asifabad,adilabad,telangana,NaN,rechini s.o,mancherial h.o,achalapur
1,ada b.o,504293,b.o,delivery,adilabad,hyderabad,andhra pradesh,asifabad,adilabad,telangana,NaN,asifabad s.o,mancherial h.o,ada
2,adegaon b.o,504307,b.o,delivery,adilabad,hyderabad,andhra pradesh,boath,adilabad,telangana,NaN,echoda s.o,adilabad h.o,adegaon
3,adilabad collectorate s.o,504001,s.o,non-delivery,adilabad,hyderabad,andhra pradesh,adilabad,adilabad,telangana,08732-226703,NaN,adilabad h.o,adilabad collectorate
4,adilabad h.o,504001,h.o,delivery,adilabad,hyderabad,andhra pradesh,adilabad,adilabad,telangana,08732-226738,NaN,NaN,adilabad


In [32]:
pindf.dtypes


officename            object
pincode                int64
officetype            object
deliverystatus        object
divisionname          object
regionname            object
circlename            object
taluk                 object
districtname          object
statename             object
telephone             object
related_suboffice     object
related_headoffice    object
dtype: object

In [27]:
pindf.select_dtypes(exclude='number')

,officename,officetype,deliverystatus,divisionname,regionname,circlename,taluk,districtname,statename,telephone,related_suboffice,related_headoffice
0,Achalapur B.O,B.O,Delivery,Adilabad,Hyderabad,Andhra Pradesh,Asifabad,Adilabad,TELANGANA,NaN,Rechini S.O,Mancherial H.O
1,Ada B.O,B.O,Delivery,Adilabad,Hyderabad,Andhra Pradesh,Asifabad,Adilabad,TELANGANA,NaN,Asifabad S.O,Mancherial H.O
2,Adegaon B.O,B.O,Delivery,Adilabad,Hyderabad,Andhra Pradesh,Boath,Adilabad,TELANGANA,NaN,Echoda S.O,Adilabad H.O
3,Adilabad Collectorate S.O,S.O,Non-Delivery,Adilabad,Hyderabad,Andhra Pradesh,Adilabad,Adilabad,TELANGANA,08732-226703,NaN,Adilabad H.O
4,Adilabad H.O,H.O,Delivery,Adilabad,Hyderabad,Andhra Pradesh,Adilabad,Adilabad,TELANGANA,08732-226738,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
154792,Uttar Sautanchak B.O,B.O,Delivery,Tamluk,South Bengal,West Bengal,Nandakumar,East Midnapore,WEST BENGAL,NaN,Mirikpur S.O,Tamluk H.O
154793,Uttarjianda B.O,B.O,Delivery,Tamluk,South Bengal,West Bengal,Panskura-i,East Midnapore,WEST BENGAL,NaN,Bhogpur S.O (East Midnapore),Tamluk H.O
154794,Uttarkotebarh B.O,B.O,Delivery,Tamluk,South Bengal,West Bengal,Bhagawanpur,East Midnapore,WEST BENGAL,NaN,Kajlagarh S.O,Tamluk H.O
154795,Uttarmechogram B.O,B.O,Delivery,Tamluk,South Bengal,West Bengal,Panskura-i,East Midnapore,WEST BENGAL,NaN,Panskura S.O,Tamluk H.O


# BELOW AREA IS DIFFERENT

In [13]:
plotly.offline.init_notebook_mode(connected=True)

In [14]:
fig = go.Figure(data=go.Bar(y=customer.data_origin.head(190)))
fig.show()

In [15]:
import plotly.graph_objects as go
fig = go.FigureWidget(data=go.Bar(y=customer.data_origin.head(190)))
fig

FigureWidget({
    'data': [{'type': 'bar',
              'uid': '26daca42-b165-44ed-905f-7d65dfc1ef0d',
     …

In [108]:
def CleanCustomerDataSet(df):
    df=df.drop(['marital_status','occupation','date_of_birth','death_date','titlea'])
    
    #impute partner_type
    df['partner_type'] = df["partner_type"].fillna(1)
    
    #title not needed
    df['title']
    

In [69]:
invoice.head()

,Amt Rcvd From Custom,Amt Rcvd From Ins Co,Area / Locality,CGST(14%),CGST(2.5%),CGST(6%),CGST(9%),CITY,Cash /Cashless Type,Claim No.,Cust Type,Customer No.,District,Expiry Date,Gate Pass Date,Gate Pass Time,IGST(12%),IGST(18%),IGST(28%),IGST(5%),Insurance Company,Invoice Date,Invoice No,Invoice Time,Job Card No,JobCard Date,JobCard Time,KMs Reading,Labour Total,Make,Misc Total,Model,ODN No.,OSL Total,Order Type,Outstanding Amt,Parts Total,Pin code,Plant,Plant Name1,Policy no.,Print Status,Recovrbl Exp,Regn No,SGST/UGST(14%),SGST/UGST(2.5%),SGST/UGST(6%),SGST/UGST(9%),Service Advisor Name,TDS amount,Technician Name,Total Amt Wtd Tax.,Total CGST,Total GST,Total IGST,Total SGST/UGST,Total Value,User ID
0,0.0,0.0,MAJIWADA,0.0,0.0,0.0,0.0,Thane,NaN,NaN,Retail,67849,Maharashtra,NaN,NaN,00:00:00,0.0,0.0,0.0,0.0,NaN,2015-01-02,7005200002,11:30:36,168303,2014-12-13,14:29:43,49317,1203.14,GENERAL MOTORS,0.00,SPARK,7.005200e+09,500.06,Paid Service,0.0,2348.75,400601,BC01,THANE,NaN,NO,0.0,KA19MA1291,0.0,0.0,0.0,0.0,NaN,0.0,RUPESH,4051.95,0.0,0.0,0.0,0.0,4051.95,BC01FS1
1,0.0,0.0,THNAE,0.0,0.0,0.0,0.0,THNAE,NaN,NaN,Retail,84419,Maharashtra,NaN,NaN,00:00:00,0.0,0.0,0.0,0.0,NaN,2015-01-03,7005200003,10:07:32,173997,2015-01-02,14:12:18,78584,804.26,TATA MOTORS,197.03,INDICA,7.005200e+09,0.00,SMC Value Package,0.0,0.00,400603,BC01,THANE,NaN,NO,0.0,MH43R3046,0.0,0.0,0.0,0.0,NaN,0.0,PRASHANT,1001.29,0.0,0.0,0.0,0.0,1001.29,BC01SA2
2,0.0,0.0,THANE,0.0,0.0,0.0,0.0,THANE[W],NaN,NaN,Retail,81055,Maharashtra,NaN,NaN,00:00:00,0.0,0.0,0.0,0.0,NaN,2015-01-03,7005200004,11:12:57,173889,2015-01-02,11:40:44,33985,180.19,MARUTI SUZUKI,0.00,ZEN,7.005200e+09,0.00,Running Repairs,0.0,52.95,400607,BC01,THANE,NaN,NO,0.0,AP09AX0582,0.0,0.0,0.0,0.0,NaN,0.0,IMRAN,233.14,0.0,0.0,0.0,0.0,233.14,BC01SA2
3,0.0,0.0,THNAE,0.0,0.0,0.0,0.0,THNAE,NaN,NaN,Retail,84419,Maharashtra,NaN,NaN,00:00:00,0.0,0.0,0.0,0.0,NaN,2015-01-03,7005200005,11:40:44,174158,2015-01-03,10:12:32,78584,0.00,TATA MOTORS,0.00,INDICA,7.005200e+09,0.00,SMC Redemption,0.0,0.00,400603,BC01,THANE,NaN,NO,0.0,MH43R3046,0.0,0.0,0.0,0.0,NaN,0.0,PRASHANT,0.00,0.0,0.0,0.0,0.0,0.00,BC01SA2
4,0.0,0.0,THANE,0.0,0.0,0.0,0.0,THANE,NaN,NaN,Retail,18980,Maharashtra,NaN,NaN,00:00:00,0.0,0.0,0.0,0.0,NaN,2015-01-03,7005200006,12:07:17,173860,2015-01-02,10:45:42,50057,1202.97,HYUNDAI,100.00,SANTRO XING,7.005200e+09,499.99,Paid Service,0.0,944.16,400603,BC01,THANE,NaN,NO,0.0,MH04CD9768,0.0,0.0,0.0,0.0,NaN,0.0,DAYANAND,2747.12,0.0,0.0,0.0,0.0,2747.12,BC01SA2


In [73]:
customer.head()

,Business Partner,Customer No.,Partner Type,Data Origin,Title,Marital Status,Occupation,Date of Birth,Death date
0,31,1,1.0,Z001,NaN,NaN,NaN,NaN,NaT
1,32,2,1.0,Z005,NaN,NaN,NaN,NaN,NaT
2,34,4,1.0,Z005,NaN,2.0,NaN,NaN,NaT
3,35,5,1.0,Z005,NaN,NaN,NaN,NaN,NaT
4,42,7,1.0,Z008,NaN,NaN,NaN,NaN,NaT


In [74]:
invoice.shape

(492314, 58)

In [75]:
invoice.tail() 

,Amt Rcvd From Custom,Amt Rcvd From Ins Co,Area / Locality,CGST(14%),CGST(2.5%),CGST(6%),CGST(9%),CITY,Cash /Cashless Type,Claim No.,Cust Type,Customer No.,District,Expiry Date,Gate Pass Date,Gate Pass Time,IGST(12%),IGST(18%),IGST(28%),IGST(5%),Insurance Company,Invoice Date,Invoice No,Invoice Time,Job Card No,JobCard Date,JobCard Time,KMs Reading,Labour Total,Make,Misc Total,Model,ODN No.,OSL Total,Order Type,Outstanding Amt,Parts Total,Pin code,Plant,Plant Name1,Policy no.,Print Status,Recovrbl Exp,Regn No,SGST/UGST(14%),SGST/UGST(2.5%),SGST/UGST(6%),SGST/UGST(9%),Service Advisor Name,TDS amount,Technician Name,Total Amt Wtd Tax.,Total CGST,Total GST,Total IGST,Total SGST/UGST,Total Value,User ID
984736,NaN,NaN,.,NaN,NaN,NaN,NaN,Chennai,NaN,NaN,Retail,203820,Tamil Nadu,NaN,NaN,00:00:00,NaN,NaN,NaN,NaN,NaN,2016-06-30,7231000017,14:56:58,400100,2016-06-30,09:15:12,72546,2495.90,MARUTI SUZUKI,374.97,ALTO,NaN,0.0,Paid Service,NaN,1587.28,600058,X405,X405 - NANGAN MOTORS PVT. LTD.,NaN,NO,0.0,TN38Y9343,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4458.15,NaN,NaN,NaN,NaN,NaN,X405SA1
984737,NaN,NaN,.,NaN,NaN,NaN,NaN,ADAMBAKKAM,NaN,NaN,Retail,170411,Tamil Nadu,NaN,NaN,00:00:00,NaN,NaN,NaN,NaN,NaN,2016-06-30,7231000018,15:06:52,400092,2016-06-30,08:42:15,18968,2443.47,NISSAN,0.00,MICRA,NaN,0.0,Paid Service,NaN,4388.61,600088,X405,X405 - NANGAN MOTORS PVT. LTD.,NaN,NO,0.0,TN22CK0396,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6832.08,NaN,NaN,NaN,NaN,NaN,X405SA1
984738,NaN,NaN,CHENNAI,NaN,NaN,NaN,NaN,CHENNAI,NaN,NaN,Retail,48811,Tamil Nadu,NaN,NaN,00:00:00,NaN,NaN,NaN,NaN,NaN,2016-06-30,7231000019,15:43:56,400082,2016-06-30,08:27:51,177433,2598.92,MAHINDRA & MAHINDRA,0.00,SCORPIO CRDE,NaN,0.0,Paid Service,NaN,6588.79,600044,X405,X405 - NANGAN MOTORS PVT. LTD.,NaN,NO,0.0,TN07BA0211,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9187.71,NaN,NaN,NaN,NaN,NaN,X405SA1
984739,NaN,NaN,"DK NAGAR EXTN,",NaN,NaN,NaN,NaN,"Vijaya Nagar, 1 main road,Velacherr",NaN,NaN,Retail,204067,Tamil Nadu,NaN,NaN,00:00:00,NaN,NaN,NaN,NaN,NaN,2016-06-30,7231000020,15:58:59,400545,2016-06-30,12:53:48,22932,11000.00,HONDA,100.00,AMAZE,NaN,0.0,Accidental,NaN,0.00,600042,X405,X405 - NANGAN MOTORS PVT. LTD.,NaN,NO,0.0,TN07BV9290,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11100.00,NaN,NaN,NaN,NaN,NaN,X405SA1
984740,NaN,NaN,2ND FLOOR,NaN,NaN,NaN,NaN,saidapet,NaN,NaN,Retail,204288,Tamil Nadu,NaN,NaN,00:00:00,NaN,NaN,NaN,NaN,NaN,2016-06-30,7231000021,17:09:19,400891,2016-06-30,16:54:06,149197,747.78,MARUTI SUZUKI,0.00,WAGONR,NaN,0.0,Running Repairs,NaN,0.00,600016,X405,X405 - NANGAN MOTORS PVT. LTD.,NaN,NO,0.0,TN10T3317,NaN,NaN,NaN,NaN,NaN,NaN,NaN,747.78,NaN,NaN,NaN,NaN,NaN,X405WM1


In [76]:
jtd.head()

,DBM Order,Order Item,Material,Labor Value Number,Description,Item Category,Order Quantity,Target quantity UoM,Net value
0,22385,180,OSLCC_SRV31,NaN,MACHINING RR.SUSPENSION CHARGE (MAJOR),P010,1.2,MIN,1869.87
1,23910,10,1DM279018170117,NaN,ASSY.OIL COOLER (6-PLATE) (VOR,P002,1.0,EA,2151.13
2,23910,40,1IS150203BE01,NaN,FRONT WHEEL BEARING/ LUBRICATE,P002,1.0,EA,2552.92
3,23910,60,1DM279001990173,NaN,ASSY.CYL.HEAD W /O INJ.(NA W /O,P002,1.0,EA,20360.14
4,23910,70,1DM88552790S001SA,NaN,Engine Overhaul Kit -Indica Di,P002,1.0,EA,10691.63


In [77]:
customer_merge_invoice_merge_jtd = pd.merge(customer_merge_invoice, jtd, left_on='Job Card No', right_on='DBM Order')

# Exploratory Data Analysis

In [78]:
customer_merge_invoice_merge_jtd.shape

(154068, 75)

In [79]:
print(customer_merge_invoice_merge_jtd.isna().sum())

Business Partner             0
Customer No.                 0
Partner Type             13596
Data Origin              29182
Title                   134818
Marital Status          154068
Occupation              154068
Date of Birth           154068
Death date              154068
Amt Rcvd From Custom      9461
Amt Rcvd From Ins Co      9461
Area / Locality         150799
CGST(14%)                 9461
CGST(2.5%)                9461
CGST(6%)                  9461
CGST(9%)                  9461
CITY                         0
Cash /Cashless Type     146584
Claim No.               152462
Cust Type                    0
District                     0
Expiry Date             153117
Gate Pass Date          142356
Gate Pass Time               0
IGST(12%)                 9461
IGST(18%)                 9461
IGST(28%)                 9461
IGST(5%)                  9461
Insurance Company       153079
Invoice Date                 0
Invoice No                   0
Invoice Time                 0
Job Card

In [80]:
customer_merge_invoice_merge_jtd.isnull().sum() / customer_merge_invoice_merge_jtd.shape[0] * 100.00

Business Partner          0.000000
Customer No.              0.000000
Partner Type              8.824675
Data Origin              18.940987
Title                    87.505517
Marital Status          100.000000
Occupation              100.000000
Date of Birth           100.000000
Death date              100.000000
Amt Rcvd From Custom      6.140795
Amt Rcvd From Ins Co      6.140795
Area / Locality          97.878210
CGST(14%)                 6.140795
CGST(2.5%)                6.140795
CGST(6%)                  6.140795
CGST(9%)                  6.140795
CITY                      0.000000
Cash /Cashless Type      95.142405
Claim No.                98.957603
Cust Type                 0.000000
District                  0.000000
Expiry Date              99.382740
Gate Pass Date           92.398162
Gate Pass Time            0.000000
IGST(12%)                 6.140795
IGST(18%)                 6.140795
IGST(28%)                 6.140795
IGST(5%)                  6.140795
Insurance Company   

In [81]:
def missing_values_percent(df: pd.DataFrame) -> pd.Series:
    """Returns percentage of missing values in each column"""
    percent_null = (df.isnull().sum() / df.shape[0]) * 100
    return pd.Series(percent_null, index=df.columns)

In [82]:
missing_values_percent(customer_merge_invoice_merge_jtd)

Business Partner          0.000000
Customer No.              0.000000
Partner Type              8.824675
Data Origin              18.940987
Title                    87.505517
Marital Status          100.000000
Occupation              100.000000
Date of Birth           100.000000
Death date              100.000000
Amt Rcvd From Custom      6.140795
Amt Rcvd From Ins Co      6.140795
Area / Locality          97.878210
CGST(14%)                 6.140795
CGST(2.5%)                6.140795
CGST(6%)                  6.140795
CGST(9%)                  6.140795
CITY                      0.000000
Cash /Cashless Type      95.142405
Claim No.                98.957603
Cust Type                 0.000000
District                  0.000000
Expiry Date              99.382740
Gate Pass Date           92.398162
Gate Pass Time            0.000000
IGST(12%)                 6.140795
IGST(18%)                 6.140795
IGST(28%)                 6.140795
IGST(5%)                  6.140795
Insurance Company   

In [83]:
customer_merge_invoice_merge_jtd.columns

Index(['Business Partner', 'Customer No.', 'Partner Type', 'Data Origin', 'Title', 'Marital Status', 'Occupation', 'Date of Birth', 'Death date', 'Amt Rcvd From Custom', 'Amt Rcvd From Ins Co', 'Area / Locality', 'CGST(14%)', 'CGST(2.5%)', 'CGST(6%)', 'CGST(9%)', 'CITY', 'Cash /Cashless Type', 'Claim No.', 'Cust Type', 'District', 'Expiry Date', 'Gate Pass Date', 'Gate Pass Time', 'IGST(12%)', 'IGST(18%)', 'IGST(28%)', 'IGST(5%)', 'Insurance Company', 'Invoice Date', 'Invoice No', 'Invoice Time', 'Job Card No', 'JobCard Date', 'JobCard Time', 'KMs Reading', 'Labour Total', 'Make', 'Misc Total', 'Model', 'ODN No.', 'OSL Total', 'Order Type', 'Outstanding Amt', 'Parts Total', 'Pin code', 'Plant', 'Plant Name1', 'Policy no.', 'Print Status', 'Recovrbl Exp', 'Regn No', 'SGST/UGST(14%)', 'SGST/UGST(2.5%)', 'SGST/UGST(6%)', 'SGST/UGST(9%)', 'Service Advisor Name', 'TDS amount', 'Technician Name', 'Total Amt Wtd Tax.', 'Total CGST', 'Total GST', 'Total IGST', 'Total SGST/UGST',
       'Total 

In [89]:
customer_merge_invoice_merge_jtd.dtypes

business_partner                object
customer_no                     object
partner_type                   float64
data_origin                     object
title                          float64
marital_status                 float64
occupation                     float64
date_of_birth                   object
death_date              datetime64[ns]
amt_rcvd_from_custom           float64
amt_rcvd_from_ins_co           float64
area__locality                  object
cgst14                         float64
cgst25                         float64
cgst6                          float64
cgst9                          float64
city                            object
cash_cashless_type              object
claim_no                        object
cust_type                       object
district                        object
expiry_date                     object
gate_pass_date                  object
gate_pass_time                  object
igst12                         float64
igst18                   

In [86]:
cleaned_df = data_cleaning(customer_merge_invoice_merge_jtd)

In [87]:
missing_values_percent(cleaned_df)
#cleaned_df.columns

business_partner          0.000000
customer_no               0.000000
partner_type              8.824675
data_origin              18.940987
title                    87.505517
marital_status          100.000000
occupation              100.000000
date_of_birth           100.000000
death_date              100.000000
amt_rcvd_from_custom      6.140795
amt_rcvd_from_ins_co      6.140795
area__locality           97.878210
cgst14                    6.140795
cgst25                    6.140795
cgst6                     6.140795
cgst9                     6.140795
city                      0.000000
cash_cashless_type       95.142405
claim_no                 98.957603
cust_type                 0.000000
district                  0.000000
expiry_date              99.382740
gate_pass_date           92.398162
gate_pass_time            0.000000
igst12                    6.140795
igst18                    6.140795
igst28                    6.140795
igst5                     6.140795
insurance_company   

In [37]:
cleaned_df['unnamed:_0_x']

0         787293
1         693968
2         693968
3         693968
4         725171
           ...  
154063    839305
154064    839305
154065    881446
154066    881446
154067    878396
Name: unnamed:_0_x, Length: 154068, dtype: int64

In [39]:
cleaned_df['unnamed:_0_x'].shape  , cleaned_df['unnamed:_0_x'].unique()

((154068,),
 array([787293, 693968, 725171, ..., 839305, 881446, 878396], dtype=int64))